In [124]:
import pandas as pd
import numpy as np
import random
import time
import copy

In [175]:
FASERIP = {"F":"Ty","A":"Ty","S":"Ty","E":"Ty","R":"Ty","I":"Ty","P":"Ty"}

dict_faserip={"Sh0":0,"Fb":2,"Pr":4,"Ty":6,"Gd":10,"Ex":20,"Rm":30,"In":40,"Am":50,"Mn":75,"Un":100,"ShX":150,"ShY":200,"ShZ":500,"Cl1000":1000,"Cl3000":3000,"Cl5000":5000,"Beyond":1000000000}

faserip_index={"Sh0":0,"Fb":1,"Pr":2,"Ty":3,"Gd":4,"Ex":5,"Rm":6,"In":7,"Am":8,"Mn":9,"Un":10,"ShX":11,"ShY":12,"ShZ":13,"Cl1000":14,"Cl3000":15,"Cl5000":16,"Beyond":17}

def column_shift(rank, shift):
	rank_list = []
	for key in faserip_index.keys():
		rank_list.append(key)
	rank_index = faserip_index[rank]
	new_index = rank_index + shift
	if new_index < 0: #lowest is zero
		new_index = 0
	if new_index > 13: #keep under cosmic
		new_index = 13
		
	return rank_list[new_index] #give back adjusted rank


def random_FASERIP():
    ability_roll = random.randint(0,6)
    ability_choice = list(FASERIP.keys())[ability_roll]
    return ability_choice

In [87]:
upbformstable = pd.read_csv('upbformstable.txt',sep='\t', skiprows=1, header=None, names=['roll','form'])
upbformstable

,roll,form
0,1,Normal human
1,2,Normal human
2,3,Normal human
3,4,Normal human
4,5,Normal human
...,...,...
95,96,Energy
96,97,Ethereal
97,98,Undead
98,99,Mutant - Breed


In [11]:
upb_form = upbformstable['form'].sample()
upb_form

38    Android
Name: form, dtype: object

In [196]:
upbnooftalentstable = pd.read_csv('upbnooftalents.txt',sep='\t', skiprows=1, header=None, names=['roll','talents_min','talents_max'])
upbnooftalentstable 

,roll,talents_min,talents_max
0,0,6,8
1,1,0,3
2,2,0,3
3,3,0,3
4,4,0,3
...,...,...,...
95,95,4,8
96,96,4,8
97,97,4,8
98,98,5,6


In [197]:
upbnoofcontactstable = pd.read_csv('upbnoofcontacts.txt',sep='\t', skiprows=1, header=None, names=['roll','contacts_min','contacts_max'])
upbnoofcontactstable 

,roll,contacts_min,contacts_max
0,0,6,6
1,1,0,2
2,2,0,2
3,3,0,2
4,4,0,2
...,...,...,...
95,95,4,5
96,96,4,5
97,97,4,5
98,98,5,5


In [89]:
upbnoofpowerstable = pd.read_csv('upbnoofpowers.txt',sep='\t', skiprows=1, header=None, names=['roll','powers_min','powers_max'])
upbnoofpowerstable

,roll,powers_min,powers_max
0,0,14,16
1,1,1,3
2,2,1,3
3,3,1,3
4,4,1,3
...,...,...,...
95,95,10,12
96,96,10,12
97,97,10,12
98,98,12,14


In [16]:
upb_powers = upbnoofpowerstable.sample()
upb_powers

,roll,powers_min,powers_max
28,28,3,5


In [198]:
upb_talents = upbnooftalentstable.sample()
upb_talents

,roll,talents_min,talents_max
64,64,2,6


In [199]:
upb_contacts = upbnoofcontactstable.sample()
upb_contacts

,roll,contacts_min,contacts_max
90,90,4,4


In [90]:
upbpowercategories = pd.read_csv('upbpowercategories.txt',sep='\s+', header=None, names=['roll','category'] )
upbpowercategories

,roll,category
0,0,17000
1,1,1000
2,2,1000
3,3,1000
4,4,1000
...,...,...
95,95,17000
96,96,17000
97,97,17000
98,98,17000


In [91]:
upbpowerlist = pd.read_csv('upbpowerlist.txt', names=['test'], header=None,engine="python")
upbpowerlist

,test
0,1001 1 0 Body Armor
1,1002 1 0 Body Armor
2,1003 1 0 Body Armor
3,1004 1 0 Body Armor
4,1005 1 0 Body Armor
...,...
1695,17096 1 0 Water Walking
1696,17097 1 0 Water Walking
1697,17098 1 0 Whirlwind
1698,17099 1 0 Whirlwind


In [92]:
upbpowerlist[['roll','c','z','power']] = upbpowerlist["test"].str.split(" ", 3, expand=True)

upbpowerlist[['roll']] = upbpowerlist[['roll']].astype('int')
upbpowerlist[['c']] = upbpowerlist[['c']].astype('int')
upbpowerlist[['z']] = upbpowerlist[['z']].astype('int')

upbpowerlist.head(32)

,test,roll,c,z,power
0,1001 1 0 Body Armor,1001,1,0,Body Armor
1,1002 1 0 Body Armor,1002,1,0,Body Armor
2,1003 1 0 Body Armor,1003,1,0,Body Armor
3,1004 1 0 Body Armor,1004,1,0,Body Armor
4,1005 1 0 Body Armor,1005,1,0,Body Armor
5,1006 1 0 Body Armor,1006,1,0,Body Armor
6,1007 1 0 Body Armor,1007,1,0,Body Armor
7,1008 1 0 Body Armor,1008,1,0,Body Armor
8,1009 1 0 Body Armor,1009,1,0,Body Armor
9,1010 1 0 Body Armor,1010,1,0,Body Armor


In [76]:
#get the list where >= the number and <= number + 1000
power_categories = []
powers_list = []

for p in range(upb_powers['powers_min'].values[0]):
#upb_upbpowercategories = upbpowercategories.sample()  #need one for each power
#upbpowercategories
    print(p)
    c = upbpowercategories.sample()
    power_categories.append(c['category'].values[0])
    #power_subset = upbpowerlist.loc[(upbpowerlist['roll'] >= c['category'].values[0])]        
    power_subset = upbpowerlist.loc[(upbpowerlist['roll'] >= c['category'].values[0]) & (upbpowerlist['roll'] < (c['category'].values[0] + 1000))]    
    #print(power_subset.shape)
    powers_list.append(power_subset.sample())

0
1
2


In [77]:
print(power_categories)
print(powers_list)

[1000, 10000, 13000]
[                  test  roll  c  z       power
5  1006 1 0 Body Armor  1006  1  0  Body Armor,                           test   roll  c  z            power
940  10041 2 0 Disintegration*  10041  2  0  Disintegration*,                           test   roll  c  z           power
1253  13054 1 0 Hyper-Strength  13054  1  0  Hyper-Strength]


In [202]:
upbtalentcategories = pd.read_csv('utbtalentcategories.txt',sep='\s+', header=None, names=['roll','category'] )
upbtalentcategories

,roll,category
0,0,18000
1,1,1000
2,2,1000
3,3,1000
4,4,1000
...,...,...
95,95,18000
96,96,18000
97,97,18000
98,98,18000


In [204]:
upbtalentlist = pd.read_csv('utbtalentlist.txt', names=['test'], header=None,engine="python")
upbtalentlist

,test
0,1001 1 Alien Studies
1,1002 1 Alien Studies
2,1003 1 Alien Studies
3,1004 1 Alien Studies
4,1005 1 Alien Studies
...,...
1795,18096 2 Weapons Specialist*
1796,18097 2 Weapons Specialist*
1797,18098 2 Weapons Specialist*
1798,18099 2 Weapons Specialist*


In [206]:
upbtalentlist[['roll','c','talent']] = upbtalentlist["test"].str.split(" ", 2, expand=True)

upbtalentlist[['roll']] = upbtalentlist[['roll']].astype('int')
upbtalentlist[['c']] = upbtalentlist[['c']].astype('int')

upbtalentlist.head(1)

,test,roll,c,talent
0,1001 1 Alien Studies,1001,1,Alien Studies


In [207]:
#get the list where >= the number and <= number + 1000
talent_categories = []
talent_list = []

for p in range(upb_talents['talents_min'].values[0]):
#upb_upbpowercategories = upbpowercategories.sample()  #need one for each power
#upbpowercategories
    print(p)
    c = upbtalentcategories.sample()
    talent_categories.append(c['category'].values[0])
    talent_subset = upbtalentlist.loc[(upbtalentlist['roll'] >= c['category'].values[0]) & (upbtalentlist['roll'] < (c['category'].values[0] + 1000))]    
    #print(power_subset.shape)
    talent_list.append(talent_subset.sample())

0
1


In [209]:
upbcontactcategories = pd.read_csv('contactsdropdownlist.txt', names=['contact'], header=None,engine="python")
upbcontactcategories

,contact
0,Medicine
1,Law
2,Law-Enforcement
3,Military
4,BusinessWorld
5,Journalism
6,Crime
7,Engineering
8,Psychiatry
9,Detective/Espionage


In [214]:
contact_list = []

for p in range(upb_contacts['contacts_min'].values[0]):
    print(p)
    c = upbcontactcategories.sample()
    contact_list.append(str(c.values[0][0]))
    
contact_list    

0
1
2
3


['Law', 'Journalism', 'Military', 'Electronics']

In [179]:
column_roll = pd.read_csv('formabilitycolumns.csv')
column_use = column_roll.loc[column_roll['Type'] == upb_form.values[0]]['Column'].values[0]
print(column_roll.head(1))
column_use

   Roll          Type  Column   1   2   3   4   5  AMT  F  A  S  E  R  I  P  \
0     1  Normal human       2  Fb  Fb  Fb  Fb  Fb   -1  0  0  0  0  0  0  0   

   Res  Pop  
0    1    0  


4

In [98]:
#want to test the forms
for ability in FASERIP.keys():
    FASERIP[ability] = column_roll[str(column_use)].sample().values[0]
    
FASERIP    

{'F': 'Am', 'A': 'In', 'S': 'Pr', 'E': 'Gd', 'R': 'Ex', 'I': 'Rm', 'P': 'Gd'}

In [101]:
all_forms = upbformstable['form'].unique()

In [102]:
all_forms

array(['Normal human', 'Mutant - Induced', 'Mutant - Random',
       'Mutant - Breed', 'Android', 'Humanoid Race', 'Surgical Composite',
       'Modified Human - Organic', 'Modified Human - Muscular',
       'Modified Human - Skeletal', 'Modified Human - Extra Parts',
       'Demihuman - Centaur', 'Demihuman - Equiman', 'Demihuman - Faun',
       'Demihuman - Felinoid', 'Demihuman - Lupinoid',
       'Demihuman - Avian', 'Demihuman - Chiropteran',
       'Demihuman - Lamian', 'Demihuman - Merhuman', 'Demihuman - Harpy',
       'Cyborg - Artificial limbs/organs', 'Cyborg - Exoskeletons',
       'Cyborg - Mechanical Body', 'Cyborg - Mechanically Augmented',
       'Robot - Human Shape', 'Robot - Usuform', 'Robot - Metamorphic',
       'Robot - Computer', 'Angel/Demon', 'Deity', 'Animal', 'Vegetable',
       'Abnormal Chemistry', 'Mineral', 'Gaseous', 'Liquid', 'Energy',
       'Ethereal', 'Undead'], dtype=object)

In [193]:
FASERIP_adjust = {}
for ability in FASERIP.keys():
    FASERIP_adjust[ability] = 0
FASERIP_adjust['H'] = 1   
FASERIP_adjust['K'] = 1   
FASERIP_adjust['Res'] = 0   
FASERIP_adjust['Pop'] = 0
FASERIP_adjust['Powers'] = 0  #becomes a list of stuff eventually - or dictionary?
FASERIP_adjust['Talents'] = 0  #becomes a list of stuff eventually - or dictionary?
FASERIP_adjust['Contacts'] = 0  #becomes a list of stuff eventually - or dictionary?
FASERIP_adjust['Weaknesses'] = {}
FASERIP_adjust['Powers_Form'] = {}
FASERIP_adjust['Spd'] = 0
FASERIP_adjust['Att'] = 0
FASERIP_adjust['Dam'] = 0
FASERIP_adjust['Climbing'] = 0
FASERIP_adjust['Escaping'] = 0

FASERIP_adjust

{'F': 0,
 'A': 0,
 'S': 0,
 'E': 0,
 'R': 0,
 'I': 0,
 'P': 0,
 'H': 1,
 'K': 1,
 'Res': 0,
 'Pop': 0,
 'Powers': 0,
 'Contacts': 0,
 'Talents': 0,
 'Weaknesses': {},
 'Powers_Form': {},
 'Att': 0,
 'Dam': 0,
 'Spd': 0,
 'Climbing': 0,
 'Escaping': 0}

In [194]:
all_forms_FASERIP = {}
for form in all_forms:
    all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
    
all_forms_FASERIP[all_forms[0]]    

{'F': 0,
 'A': 0,
 'S': 0,
 'E': 0,
 'R': 0,
 'I': 0,
 'P': 0,
 'H': 1,
 'K': 1,
 'Res': 0,
 'Pop': 0,
 'Powers': 0,
 'Contacts': 0,
 'Talents': 0,
 'Weaknesses': {},
 'Powers_Form': {},
 'Att': 0,
 'Dam': 0,
 'Spd': 0,
 'Climbing': 0,
 'Escaping': 0}

In [208]:
ability_choice = str(random_FASERIP())

##adjustments
form = 'Normal human'
all_forms_FASERIP[form]['Res'] = 1

form = 'Mutant - Induced'
all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
all_forms_FASERIP[form][ability_choice] = 1
all_forms_FASERIP[form]
form = 'Mutant - Random'
all_forms_FASERIP[form]['E'] = 1
all_forms_FASERIP[form]['Res'] = -1
all_forms_FASERIP[form]['Powers'] = 1
all_forms_FASERIP[form]
form = 'Mutant - Breed'
all_forms_FASERIP[form]['E'] = 1
all_forms_FASERIP[form]['I'] = 1
all_forms_FASERIP[form]['Contacts'] = 1

form = 'Android'
all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
all_forms_FASERIP[form][ability_choice] = 1
all_forms_FASERIP[form]['Pop'] = -1
all_forms_FASERIP[form]['Powers'] = 1
all_forms_FASERIP[form]['Contacts'] = 1

form = 'Humanoid Race'
all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
all_forms_FASERIP[form][ability_choice] = 1
all_forms_FASERIP[form]['Contacts'] = 1
all_forms_FASERIP[form]['Powers'] = 1
all_forms_FASERIP[form]['Res'] = {'Max':'Pr'}

form = 'Surgical Composite'
all_forms_FASERIP[form]['F'] = 1
all_forms_FASERIP[form]['S'] = 1
all_forms_FASERIP[form]['E'] = 1
all_forms_FASERIP[form]['Contacts'] = 1
all_forms_FASERIP[form]['Res'] = {'Max':'Pr'}
all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}
all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}
all_forms_FASERIP[form]['Powers_Form'] = {"Regeneration":"Fb"}
all_forms_FASERIP[form]['Weaknesses'] = {"Resist Mental Domination":-1}
all_forms_FASERIP[form]

form = 'Modified Human - Organic'
all_forms_FASERIP[form]['Powers_Form'] = {"Regeneration":"Fb"}

form = 'Modified Human - Muscular'
all_forms_FASERIP[form]['S'] = 1
all_forms_FASERIP[form]['E'] = 1
all_forms_FASERIP[form]['Powers'] = -1

form = 'Modified Human - Skeletal'
all_forms_FASERIP[form]['Powers'] = -1
all_forms_FASERIP[form]['Powers_Form'] = {"Resist: Physical":"Fb"}

form = 'Modified Human - Extra Parts'
#needs a list of Extra Parts to gain abilitiies from
extra_parts = ['Arms','Tail','Wings','Fins','Antennae','Duplicate Organs']  #add powers from here
random_index = random.randrange(len(extra_parts))
extra_part = extra_parts[random_index]
all_forms_FASERIP[form]['Powers'] = -1
if extra_part == extra_parts[0]:
    all_forms_FASERIP[form]['F'] = 1
elif extra_part == extra_parts[1]:
    all_forms_FASERIP[form]['Att'] = 1
elif extra_part == extra_parts[2]:
    all_forms_FASERIP[form]['Powers_Form'] = {"True Flight":0}
elif extra_part == extra_parts[3]:
    all_forms_FASERIP[form]['Powers_Form'] = {"Hyper-Swimming":0}
elif extra_part == extra_parts[3]:
    all_forms_FASERIP[form]['Powers_Form'] = {"True Sight":0}
else:
    all_forms_FASERIP[form]['H'] = 2
#print(all_forms_FASERIP[form])

form = 'Demihuman - Centaur'
all_forms_FASERIP[form]['S'] = 1
all_forms_FASERIP[form]['Climbing'] = -2

form = 'Demihuman - Equiman'
all_forms_FASERIP[form]['Dam'] = 1

form = 'Demihuman - Faun'
all_forms_FASERIP[form]['Dam'] = 1
all_forms_FASERIP[form]['Powers_Form'] = {"Domination":"Fb"}

form = 'Demihuman - Felinoid'
all_forms_FASERIP[form]['Powers_Form'] = {"Natural Weaponry":"S","Thermal Vision":"R"}

form = 'Demihuman - Felinoid'
all_forms_FASERIP[form]['Powers_Form'] = {"Natural Weaponry":"S","Thermal Vision":"Ex"}
all_forms_FASERIP[form]['Climbing'] = 2

form = 'Demihuman - Lupinoid'
all_forms_FASERIP[form]['Pop'] = -1
all_forms_FASERIP[form]['Powers_Form'] = {"Hyper-Olfactory":"Ex"}
all_forms_FASERIP[form]['Climbing'] = 2

form = 'Demihuman - Avian'
all_forms_FASERIP[form]['Pop'] = 1
all_forms_FASERIP[form]['Powers_Form'] = {"True Flight":"Ex"}

form = 'Demihuman - Chiropteran'
all_forms_FASERIP[form]['Pop'] = {'Max':'Fb'}
all_forms_FASERIP[form]['Powers_Form'] = {"Sonar":"Gd","True Flight":"Ty"}

form = 'Demihuman - Lamian'
all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}
all_forms_FASERIP[form]['Escaping'] = 1
all_forms_FASERIP[form]

form = 'Demihuman - Merhuman'
all_forms_FASERIP[form]['Pop'] = 1
all_forms_FASERIP[form]['Powers_Form'] = {"Water Freedom":"Un"}

form = 'Demihuman - Harpy'
all_forms_FASERIP[form]['F'] = 1
all_forms_FASERIP[form]['Powers_Form'] = {"True Flight":"Ty"}

form = 'Cyborg - Artificial limbs/organs'
all_forms_FASERIP[form]['I'] = -1

form = 'Cyborg - Exoskeletons'
#got to roll on Ability modifier table for each of these
all_forms_FASERIP[form] = copy.deepcopy(FASERIP_adjust)
for ability in FASERIP.keys():
    #print(all_forms_FASERIP[form][ability])
    #print(column_roll['AMT'].sample().values[0])
    all_forms_FASERIP[form][ability] = column_roll['AMT'].sample().values[0]
#print(all_forms_FASERIP[form])
   
form = 'Cyborg - Mechanical Body'
all_forms_FASERIP[form]['I'] = -1
all_forms_FASERIP[form]['P'] = -1
all_forms_FASERIP[form]['Contacts'] = 1
all_forms_FASERIP[form]['Powers_Form'] = {"Resist: Poison":"Mn","Resist: Disease":"Mn"}

form = 'Cyborg - Mechanically Augmented'
all_forms_FASERIP[form]['Powers'] = -1
all_forms_FASERIP[form]['Res'] = {'Min':'Gd'}

form = 'Robot - Human Shape' 
all_forms_FASERIP[form]['Pop'] = {'Max':'Sh0'}

form = 'Robot - Usuform'
##no changes

form = 'Robot - Metamorphic'
##no changes #two forms, human and robot - for simulation -1CS - add this in the compund bucket

form = 'Robot - Computer'
all_forms_FASERIP[form]['F'] = -1
all_forms_FASERIP[form]['R'] = 2
all_forms_FASERIP[form]['Res'] = 1
all_forms_FASERIP[form]['Weaknesses'] = {"Resist: Electricty":-1,"Resist: Magnetism":-1,"Resist: Phasing":-1}

form = 'Angel/Demon' 
side = random.randint(1,2)
all_forms_FASERIP[form]['F'] = 1
all_forms_FASERIP[form]['A'] = 1
all_forms_FASERIP[form]['S'] = 1
all_forms_FASERIP[form]['E'] = 1
all_forms_FASERIP[form]['R'] = 1
all_forms_FASERIP[form]['I'] = 1
all_forms_FASERIP[form]['P'] = 1
if side == 1:
    all_forms_FASERIP[form]['Pop'] = 2
    all_forms_FASERIP[form]['Powers_Form'] = {"Artifact Creation-sword":"Ex"}
else:
    all_forms_FASERIP[form]['Pop'] = -2
    all_forms_FASERIP[form]['Powers_Form'] = {"Fire Generation":"Gd", "True Invulnerability: Heat":"Cl1000", "True Invulnerability: Fire":"Cl1000"}

form = 'Deity'
all_forms_FASERIP[form]['F'] = 2
all_forms_FASERIP[form]['A'] = 2
all_forms_FASERIP[form]['S'] = 2
all_forms_FASERIP[form]['E'] = 2
all_forms_FASERIP[form]['R'] = 2
all_forms_FASERIP[form]['I'] = 2
all_forms_FASERIP[form]['P'] = 2
all_forms_FASERIP[form]['Pop'] = 2

form = 'Animal' 
all_forms_FASERIP[form]['Res'] = {'Max':'Sh0'}
all_forms_FASERIP[form]['Powers'] = -1
all_forms_FASERIP[form]['Powers_Form'] = {"Hyper-Hearing":"Gd", "Tracking":"Gd"}  #as examples, need a bunch of forms again

form = 'Vegetable'
all_forms_FASERIP[form]['F'] = -2
all_forms_FASERIP[form]['E'] = 2
all_forms_FASERIP[form]['Contacts'] = 1
all_forms_FASERIP[form]['Powers_Form'] = {"Absorption:Light":"Gd"}

form = 'Abnormal Chemistry' 
all_forms_FASERIP[form]['E'] = 1

form = 'Mineral'
all_forms_FASERIP[form]['H'] = 2
all_forms_FASERIP[form]['Spd'] = -1

form = 'Gaseous' 
all_forms_FASERIP[form]['F'] = {'Max':'Sh0'}
all_forms_FASERIP[form]['Res'] = {'Max':'Sh0'}
all_forms_FASERIP[form]['Contacts'] = {'Max':'Sh0'}
all_forms_FASERIP[form]['Powers_Form'] = {"Resist:Physical":"Mn"}

form = 'Liquid'
all_forms_FASERIP[form]['Contacts'] = 1

form = 'Energy'
all_forms_FASERIP[form]['Powers_Form'] = {"Energy Field":"Fb", "Energy Emission":0, "Optional:Energy Control":0}
all_forms_FASERIP[form]['Weakness'] = {"Plasma Control":-1}

form = 'Ethereal'
all_forms_FASERIP[form]['F'] = {'Max':'Sh0'}
all_forms_FASERIP[form]['Powers_Form'] = {"Resist:Physical":"Mn"}
all_forms_FASERIP[form]['Weakness'] = {"Resist:Mental":-1,"Resist:Magical":-1}

form = 'Undead'
all_forms_FASERIP[form]['S'] = 1
all_forms_FASERIP[form]['E'] = 1
all_forms_FASERIP[form]['Weakness'] = {"Holy:Symbol":"Ex"}

all_forms_FASERIP[form]

{'F': 0,
 'A': 0,
 'S': 1,
 'E': 1,
 'R': 0,
 'I': 0,
 'P': 0,
 'H': 1,
 'K': 1,
 'Res': 0,
 'Pop': 0,
 'Powers': 0,
 'Contacts': 0,
 'Talents': 0,
 'Weaknesses': {},
 'Powers_Form': {},
 'Att': 0,
 'Dam': 0,
 'Spd': 0,
 'Climbing': 0,
 'Escaping': 0,
 'Weakness': {'Holy:Symbol': 'Ex'}}

In [218]:
#### need to do modifications here
#character needs a powers list, a contacts list, a talents list - with types perhaps as a lookup
#also need to eventually get a beastiaryFASERIP style output
#alignment - need a random name generator? otherwise some sort of id linking to sheet
#subsume all these into beastiaryFASERIP?
#check for extra powers
#check for extra contacts
print(upb_form.values[0])
adjustments = all_forms_FASERIP[upb_form.values[0]]
adjustments

Android


{'F': 0,
 'A': 0,
 'S': 1,
 'E': 0,
 'R': 0,
 'I': 0,
 'P': 0,
 'H': 1,
 'K': 1,
 'Res': 0,
 'Pop': -1,
 'Powers': 1,
 'Contacts': 1,
 'Talents': 0,
 'Weaknesses': {},
 'Powers_Form': {},
 'Att': 0,
 'Dam': 0,
 'Spd': 0,
 'Climbing': 0,
 'Escaping': 0}

In [222]:
type(adjustments[ability])

int

In [235]:
#generate FASERIP
character = copy.deepcopy(adjustments)

for ability in FASERIP.keys():
    character[ability] = column_roll[str(column_use)].sample().values[0]
    print(character[ability])
    if type(adjustments[ability]) == int:
        character[ability] = column_shift(character[ability], adjustments[ability])
        print(character[ability])
    else:
        #dict of max min
        pass
character['Res'] = "Ty"
resource_adj = column_roll['AMT'].sample().values[0]
character['Pop'] = "Ty"

character['H'] = dict_faserip[character['F']] + dict_faserip[character['A']] + dict_faserip[character['S']] + dict_faserip[character['E']]
character['K'] = dict_faserip[character['R']] + dict_faserip[character['I']] + dict_faserip[character['P']]
character['H'] = character['H'] * adjustments['H'] 

if type(adjustments['Res']) == int:
    character['Res'] = column_shift(character['Res'], adjustments['Res'] + resource_adj)
else:
    pass #dict with max and minimum

if type(adjustments['Pop']) == int:
    character['Pop'] = column_shift(character['Pop'], adjustments['Pop'] + resource_adj)
else:
    pass #dict with max and minimum

talents = []
for talent in talent_list:
    talents.append(str(talent['talent'].values[0]))
character['Talents'] = talents

if type(adjustments['Contacts']) == int:
    if adjustments['Contacts'] == 1:
        character['Contacts'] = ['Inital Form Contact']
    else:
        contacts = []
        for contact in contact_list:
            contacts.append(str(contact['conctact'].values[0]))
        character['Contacts'] = conctacts
        
else:
    character['Contacts'] = []



character                                                                                          

Rm
Rm
Fb
Fb
Gd
Ex
Gd
Gd
Ty
Ty
In
In
Am
Am


{'F': 'Rm',
 'A': 'Fb',
 'S': 'Ex',
 'E': 'Gd',
 'R': 'Ty',
 'I': 'In',
 'P': 'Am',
 'H': 62,
 'K': 96,
 'Res': 'Ty',
 'Pop': 'Pr',
 'Powers': 1,
 'Contacts': ['Inital Form Contact'],
 'Talents': ['Computer Engineering', 'Martial Arts C'],
 'Weaknesses': {},
 'Powers_Form': {},
 'Att': 0,
 'Dam': 0,
 'Spd': 0,
 'Climbing': 0,
 'Escaping': 0}

In [233]:
len(talent_list)
for x in talent_list:
    print(x)

                            test  roll  c                talent
638  7039 1 Computer Engineering  7039  1  Computer Engineering
                        test   roll  c          talent
1032  11033 1 Martial Arts C  11033  1  Martial Arts C
